# Concatonating Data

SQL provides more than one method to concatonate data, however in SQL Server some of these are implemented within different versions of the product and therefore you need to use the one most appropriate to your version.

**Hint -** *To find out your version of SQL Server, run the command "select @@version" in your SQL Server Management Studio and it will tell you accordingly.*

Within this section we will hope to solve the following part of our query:
-   Year of order
-   Day of Week order was taken
-   Employee Name
-   **Customer's Full Name**
-   Total Order Value (Sale Amount + Tax)

Currently the customer's name is spread across 3 fields and, as you will see, I've sneakily added a few NULL values into the mix in order to highlight some quirks of the methods we are about to try.

#### The Plus Symbol

The most basic method of concatonating fields is to use the + symbol.  This will work in any and all versions of SQL Server and has been around since day one.  However, it does have its quirks and pitfalls which we will see shortly but, firstly, I would be amiss if I didn't point out the largest confusion of all:

In [ ]:
select 1 + 1;

select '1' + '1';

Yes, as you can see from the above, the plus sign is exactly that, it tells SQL Server to add two numbers together.  However, if SQL thinks that you have provided it with two strings then it will simply concatonate them instead.

This causes a lot of confusion but once you have become used to datatypes and how SQL interprets them this becomes less of a concern.

Let's jump right in and simply concatonate our customer details into a full name and see what SQL returns.  Once we know the results we can then look to tailor accordingly:


In [ ]:
select customerTitle, customerFName, customerLName,
		customerTitle + customerFName + customerLName
from JupyterDatabase.sales.basicSales;

There are two immediate concerns.  Firstly we have no spaces between the concatonations, although this is to be expected as SQL is literally concatonating values and doesn't know we require spaces, and secondly that several of the results have returned as NULL.

We shall deal with the NULL values first.

In a previous section I explained that NULL values are 'unknown' and therefore cannot equate to anything, be added to anything etc., well they can also not be concatonated to anything.  Therefore although in our table we are only missing a first name, our concatonation immediately returns NULL as the result of the entire concatonation (title + NULL = NULL.  NULL + last name = NULL).

However, in the same previous section we encountered the ISNULL function which can replace a NULL value with something of our choosing, so let's utilise this here and replace our NULL values with a blank space (this would be done via the use of ''):

In [ ]:
select customerTitle, customerFName, customerLName,
		customerTitle + isnull(customerFName, '') + customerLName
from JupyterDatabase.sales.basicSales;

This is a much better result.  We have now removed the NULL values from our concatonated column.

Our remaining issue is the lack of spaces between the columns once concatonated.  We could just add these into our function:

In [ ]:
select customerTitle, customerFName, customerLName,
		customerTitle + ' ' + isnull(customerFName, '') + ' ' + customerLName
from JupyterDatabase.sales.basicSales;

This looks much better but, again, isn't quite right. If we look closely at our customers with no first name we can see that there are now two spaces between the title and last name.

So how can we cleanly deal with this?

We have to use a little complex trickery for this one. If you remember from before, if we concatonate to a NULL we are left with NULL. So let's do that inside the ISNULL function:

`isnull(customerFName + ' ', '')`

This tells us that if customerFName is null then customerFName + ' ' will still be NULL. Therefore ISNULL replaces this with a simple blank space. However, if customerFName is a value then it will concatonate a space on the end.  We therefore no longer need to add a space between first and last name.

Hence this may seem messy, but it does nicely solve our problem:

In [ ]:
select customerTitle, customerFName, customerLName,
		customerTitle + ' ' + isnull(customerFName + ' ', '') + customerLName
from JupyterDatabase.sales.basicSales;

This works knowing we may have NULL first name values, but what if we could have NULL values for our customer title and last names as well?

Now it becomes very messy indeed, but it does make our code more complete (if we have no first name or last name it will leave excess spaces, but we have to presume that wouldn't happen in real life):

In [ ]:
select customerTitle, customerFName, customerLName,
		isnull(customerTitle + ' ', '') + isnull(customerFName + ' ', '') + isnull(customerLName, '')
from JupyterDatabase.sales.basicSales;

#### CONCAT

SQL Server 2012 introduced a new function called CONCAT which aims to make this a little easier.

Sadly it isn't perfect, but I find it does make things a little easier to understand.

CONCAT works by simply letting you provide a list of strings and SQL Server will concatonate them together.  Luckily the function natively deals with NULL values, automatically replacing them with blanks so that you don't have to:

In [ ]:
select customerTitle, customerFName, customerLName,
		concat(customerTitle, customerFName, customerLName)
from JupyterDatabase.sales.basicSales;

As you can see from the results, this requires no messy ISNULL function to be coded manually as the function caters for this by default.

However, as you can see, this doesn't deal with the requirement that we might want to add spaces between our column values.  But, again, as this implicitly caters for ISNULL by itself we simply need to add the space to each column in order to solve the problem:

In [ ]:
select customerTitle, customerFName, customerLName,
		concat(customerTitle + ' ', customerFName + ' ', customerLName)
from JupyterDatabase.sales.basicSales;

Therefore you can see why this is much cleaner and the preferred method as long as you are on the relevant SQL Server version.

#### CONCAT_WS

Finally we come onto a new addition into SQL Server 2017 which makes lives a lot easier by extending the abilities of the basic CONCAT function.

<code>CONCAT_WS(separator, string, string1, string2, ..., stringN)</code>

Hence we can tell SQL Server that we wish to see spaces between our strings:

In [ ]:
select customerTitle, customerFName, customerLName,
		concat_ws(' ', customerTitle, customerFName, customerLName)
from JupyterDatabase.sales.basicSales;

This solves our problem in the most clean and complete manner with no hacks or tweaks.

Note that we can use any separator we like, for example we could choose an exclamation mark:

In [ ]:
select customerTitle, customerFName, customerLName,
		concat_ws('!', customerTitle, customerFName, customerLName)
from JupyterDatabase.sales.basicSales;